In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
n_collect = 200

In [3]:
np.linspace(0, 0.8, 33)

array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  ])

In [ ]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 0.8, 33), tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [ ]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=n_collect, tp='VulACF')
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=n_collect, tp='PlainACF')
    mss = pd.concat([mss0, mss1])
    mss_stats = pd.concat([mss_stats0, mss_stats1])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

In [ ]:
def collect_fu(pars_post, n_collect=20, coverage=0.03, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'VulACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 2}},
        'Vul_2_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 2}},
        'Vul_3_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 3}},
        'Vul_3_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 3}},
    }
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [19]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fu(pars_post, n_collect=n_collect, coverage=0.01)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_lo


NameError: name 'collect_fu' is not defined

In [ ]:
mss_stats

In [4]:
def collect_fux(pars_post, n_collect=20, coverage=0.03, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Vulfu_Footfall', 'ACF_Vulfu_Screened', 'ACF_Vulfu_Confirmed', 'ACF_Vulfu_Yield', 'ACF_Vulfu_TP',
           'ACF_Vulfu_DS_Fl', 'ACF_Vulfu_DR_Fl', 'ACF_Vulfu_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
    
        _, ms, _ = m.simulate_onward(y1, pars, intv={}, t_end=2031)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        ms_stats = {'Key': i, 'Scenario': 'Baseline', 'Pop0': ms.Pop.iloc[0], 
                    'FollowUp': 0, 'Duration': 0}
        for key in idx[1:]:
            try:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            except KeyError:
                ms_stats[key] = 0

        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            if n_fu > 0:
                fu = 1 / n_fu
            else:
                fu = 0
            for dur in np.linspace(0, 4, 9):
                intv = {'VulACF': {'Coverage': coverage, 'FollowUp': fu, 'Duration': dur}}
                
                _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
                try:
                    ms = ms.reset_index().assign(Key = i, FollowUp=fu, Duration=dur)
                    mss.append(ms)

                    ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0], 
                                'FollowUp': fu, 'Duration': dur}

                    for key in idx[1:]:
                        try:
                            ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
                        except KeyError:
                            ms_stats[key] = 0

                    mss_stats.append(ms_stats)
                except TypeError:
                    pass
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [ ]:
for folder in ['dy_hi', 'dy_lo']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fux(pars_post, n_collect=n_collect, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_fudur.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_fudur_stats.csv')

dy_hi
dy_lo


In [ ]:
mss_stats